# Magic: The Gathering card creator
<i>Multiple agents working together to meet a larger goal.</i>

## Motivation
### You can't ask Chat GPT to do it all
If you ask a good AI agent to perform a complex task, you may get something close to what you want, but not exactly right. In this case, the only way to improve the output is to modify the only input at your disposal: the Prompt. This can become very disappointing quickly.<br>
### But you can use good design practices to get where you want to go
Rather than asking one master agent to answer everything, break the problem down into distinct chunks. Simply put, they fall into two categories:
<ul><li>AI Agent: Tasks you can't do (at least easily) on your own</li>
<li>YOU: Tasks you can handle.</li>
</ul>

## Overview
For this case, we build a <b>Magic: The Gathering</b> creature card creator. Such a task involves 3 distinct efforts, which require different skills:
<ol><li>An entertaining, engaging illustration showing the creature in its environment.</li>
<li>Key features: name, type, abilities, mana cost, rarity, etc.</li>
<li>Assembling the information on one card in a layout which matches the game.</li>
</ol>
Asking a single agent to generate a card by itself effectively suffers from too great of a solution space. Further, some elements require a great deal of creativity (the illustration) and others require following specific directions (layout), and others are in between (the features list). Thus by breaking the job into 3 tasks, we expect to get better results than asking for a card all at once.</br></br>
For this demo, we begin by asking our AI agent to create a creature card in one request. Next, we break the problem into sub-tasks and try again. For fairness, we will make the prompt as similar as practical. Also, as this involves image generation which could become costly especially when troubleshooting results, we use open-source libraries whenever possible.

### AI Agents
We use 2 AI Agents: one that performs Stable Diffusion to create an image, and an LLM to determine the creature's features.
### non-AI Agent
We also use a non-AI agent which uses the Python module Pillow to do the typesetting and card assembly.

### Import libraries & Initialize AI clients and image generation pipeline

In [1]:
from typing import List, Dict, Callable
from agents.tool import WebSearchTool
from agents import Agent, Runner, trace
from agents import OpenAIChatCompletionsModel, AsyncOpenAI
from agents.model_settings import ModelSettings
from langchain.chat_models import init_chat_model
import openai
from openai import OpenAI
# Display & data parsing functions
from IPython.display import display, Markdown
from pprint import pprint
import json
# For image generation
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image, ImageDraw, ImageFont
from compel import Compel
import json, textwrap
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageOps

import os
HOME_DIR = f"{os.environ['HOME']}/Documents/GitHub/Generative-AI"
result_path = f"{HOME_DIR}/results/MTG-card-generation"
dataset_path = f"{HOME_DIR}/datasets/MTG-card-generation"
GEN_OPENAI_IMAGE = False  # Set to True to generate image using OpenAI DALL-E-3

# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv()

# Allow nested async
import nest_asyncio

# Identify gpu type (or cpu if none available)
from local_tools import get_device

# Initialize
nest_asyncio.apply()
device = get_device()

# Initialize image generation pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

# --- Client for local/OSS or OpenAI ---
client = OpenAI(
    base_url="http://localhost:11434/v1",  # adjust if needed
    api_key="ollama:gpt-oss:20b",          # dummy for OSS
)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### Create a MTG creature card which uses black mana "at once"
We will create a card using 2 methods:<ol>
<li>Stable Diffusion - an image generator using the open source library gpt-oss:20b</li>
<li>ChatGPT5</li></ol>

In [2]:
creature_prompt = """Depict a rare creature card for the game Magic: The Gathering that uses black mana."""

# First, generate an image using Stable Diffusion. If NSFW content is detected, regenerate until a safe image is produced.
# For stable diffusion it's good to have one dimension=512. Also both dimensions should be multiples of 8.
# As MTG cards are portrait orientation, we use height>width. Since the size ~2.5 inches wide, 3.5 inches high, we use 512x736 pixels.
nsfw_detected = True
while nsfw_detected:
    images = pipe(prompt=creature_prompt, guidance_scale=10, num_inference_steps=50, num_images_per_prompt=1, height=688, width=512)
    nsfw_detected = images.nsfw_content_detected[0]
card = images.images[0]
card.show()
card.save(f"{result_path}/stable_diffusion_mtg_creature_card.png")

  0%|          | 0/50 [00:00<?, ?it/s]

<img src=./datasets/MTG-card-generation/stable_diffusion_mtg_creature_card.png />
</br>
This contains the spirit of the card, but is not playable. The text is scribbles, there is no header, etc.</br>
Next, try using OpenAI

In [3]:
if GEN_OPENAI_IMAGE:
    response = openai.images.generate(
            model="dall-e-3",
            prompt=creature_prompt,
            n=1,
            size="1792x1024"
        )
    image_url = response.data[0].url
    print(f"Generated image URL: {image_url}")


<img src=./datasets/MTG-card-generation/openai_mtg_black_mana_creature.png width=800/>
</br>
Again, it's an attractive card but clearly not playable.

### Create a MTG creature card by breaking the job into pieces
#### Pre-work
Before the job, we created a template card for a black mana and green mana creature. Though it could be done programmatically, we used an existing MTG image and masked sections until we had a reasonably good template:</br>
<img src=./datasets/MTG-card-generation/mtg_black_frame_transparent.png width=400/></br>
Though the mana cost is still there, we decided to leave it in place.

#### Generate card features in JSON
The first agent MTGCardAgent uses an LLM to describe a Magic: The Gathering creature with these qualities populated in JSON:</br>
<ul><li>Name</li>
<li>Mana cost</li>
<li>Type</li>
<li>Rarity</li>
<li>Abilities</li>
<li>Power</li>
<li>Toughness</li>
<li>Flavor Text</li></ul>
In 2024, MTG cards stopped using the term "... enters the battlefield" for their creature description and instead used "enters". To ensure our description is for current cards, we filter the generated abilities to use the more current term.

In [4]:
# Card parameters -- currently can only support black or green mana
rarity = "rare"
mana = "black"

# --- Simple card generator agent ---
class MTGCardAgent:
    def __init__(self, client, model="gpt-oss:20b"):
        self.client = client
        self.model = model

    def run(self, prompt: str):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert Magic the Gathering card designer."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


# --- Generate MTG card ---
agent = MTGCardAgent(client)

prompt = f"""Design a brand-new Magic: The Gathering card template, {mana} {rarity} creature.
Return your answer in JSON with fields:
- name
- mana_cost
- type_line
- rarity
- abilities (list of strings)
- power
- toughness
- flavor_text
"""

card_json = agent.run(prompt)
# remove head/tail to keep the true JSON string. Also make sure to replace newlines and "enters the battlefield" with "enters"
card_json = card_json[card_json.index("```")+7:card_json.index("```",10)-1].replace("\n", "").replace("enters the battlefield", "enters")

card = json.loads(card_json)
pretty = json.dumps(card, indent=2)
display(Markdown(f"```json\n{pretty}\n```"))


```json
{
  "name": "Sable Reclaimer",
  "mana_cost": "3BB",
  "type_line": "Creature \u2014 Shade",
  "rarity": "Rare",
  "abilities": [
    "When Sable Reclaimer dies, return target creature card from your graveyard to your hand.",
    "Sacrifice another creature: Sable Reclaimer deals 2 damage to any target.",
    "When Sable Reclaimer enters, you may sacrifice a creature. If you do, Sable Reclaimer gets +2/+2 until end of turn."
  ],
  "power": "3",
  "toughness": "3",
  "flavor_text": "Only those who taste the abyss can claim its bounty."
}
```

#### Use the JSON input to define the artwork
We use the Stable Diffusion pipeline to create the artwork. We also add some colorful description of the environment for the creature.

In [5]:
# --- Generate artwork from parsed JSON ---
if card:
    if mana == "black":
        art_prompt = f"""
            Epic dark fantasy illustration of a Magic: The Gathering creature.
            Depict a {card['type_line']} named {card['name']}.
            Theme: {card['flavor_text']}
            Style: painterly, highly detailed, atmospheric, gothic horror.
            Lighting: dramatic shadows, eerie glow, sinister ambiance.
            Focus: single centered subject, portrait orientation.
            Do NOT include card borders, text boxes, logos, or templates.
            """
    else:
        art_prompt = f"""
            Epic fantasy illustration of a powerful green creature in Magic: The Gathering style.
            Depict a {card['type_line']} named {card['name']}.
            Theme: {card['flavor_text']}
            Setting: deep forest bathed in sunlight shafts, lush foliage, mystical atmosphere.
            Style: painterly, highly detailed, natural textures, fantasy card game art.
            Focus: single centered creature, portrait orientation.
            Do NOT include card borders, frames, text, numbers, logos, or templates.
            """

    neg_prompt = """
    card frame, border, text, captions, watermarks, logos, symbols, template,
    numbers, stats, cropped edges, trading card, design layout
    """

    # Init compel with the pipe’s tokenizer + text_encoder
    compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)

    # This auto-chunks >77 token prompts
    prompt_embeds = compel(art_prompt)
    neg_prompt_embeds = compel(neg_prompt)
    nsfw_detected = True
    while nsfw_detected:
        images = pipe(prompt_embeds=prompt_embeds, negative_prompt_embeds=neg_prompt_embeds, guidance_scale=10, num_inference_steps=50, num_images_per_prompt=1, height=512, width=624)
        nsfw_detected = images.nsfw_content_detected[0]
    art = images.images[0]
    art.show()
    art.save(f"{result_path}/art_agent_mtg_black_mana_creature.png")




  0%|          | 0/50 [00:00<?, ?it/s]

<img src=./datasets/MTG-card-generation/art_agent_mtg_black_mana_creature.png></br>
Notice that the image isn't arbitrary. It is "Ebon Harvester" which does its 'reaping' in the forest, which aligns with the picture.

#### Assembly - put the frame, artwork and creature characteristics together
As each element of a card belongs in a specific location, we use the Python Pillow library for the assembly and typesetting. This code calls out various bounding boxes in an image as well as the fonts to use for the text.

In [6]:

BLACK_FRAME_PATH = f"{dataset_path}/mtg_black_frame_transparent.png"
GREEN_FRAME_PATH = f"{dataset_path}/mtg_green_frame_transparent.png"
OUTPUT_PATH   = f"{result_path}/mtg_card_final.png"

# Font paths (adjust to where you installed them)
FONT_BELEREN_SC = "Beleren2016SmallCaps.ttf"  # name/title/type
FONT_BELEREN_B  = "Beleren2016-Bold.ttf"      # alt/title/PT
FONT_RULES      = "MPlantin.ttf"              # rules & flavor; substitute CrimsonText-SemiBold.ttf if you don't have Plantin
# Fallbacks:
FALLBACK_SANS   = "/System/Library/Fonts/Supplemental/Arial Unicode.ttf"
FALLBACK_SERIF  = "/System/Library/Fonts/Supplemental/Times New Roman.ttf"

# -----------------------------
# 1) Parameters for BOXES 
# -----------------------------
# These are measured to look right on a 744×1040 frame. Tweak if your frame template differs.
W, H = 1024, 1536
NAME_BOX  = (82, 95, 560, 130)      # left top right bottom (leaves room for mana)
MANA_BOX  = (570, 95, 850, 130)
ART_BOX   = (82, 140, 950, 850)
TYPE_BOX  = (82, 860, 880, 900)
TEXT_BOX  = (82, 935, 880, 1330)     # rules + flavor
PT_BOX    = (740, 1265, 1015, 1415)   # power/toughness
RARITY_C  = (900, 880)               # small circle center (title bar right)


# -----------------------------
# 2) HELPERS
# -----------------------------
def load_font(path, size, fallback=None):
    try:
        return ImageFont.truetype(path, size)
    except Exception:
        if fallback:
            try:
                return ImageFont.truetype(fallback, size)
            except Exception:
                return ImageFont.load_default()
        return ImageFont.load_default()

def wrap_text_by_width(draw, text, font, max_width):
    lines = []
    for paragraph in text.split("\n"):
        words = paragraph.split(" ")
        line = ""
        for w in words:
            test_line = line + (" " if line else "") + w
            if draw.textlength(test_line, font=font) <= max_width:
                line = test_line
            else:
                lines.append(line)
                line = w
        if line:
            lines.append(line)
    return lines

def fit_image_to_box(img: Image.Image, box):
    """Crop (letterbox) to preserve aspect ratio and fill the box exactly."""
    box_w = box[2] - box[0]
    box_h = box[3] - box[1]
    return ImageOps.fit(img, (box_w, box_h), method=Image.LANCZOS, bleed=0.0, centering=(0.5, 0.5))

def shrink_to_fit(draw: ImageDraw.ImageDraw, text, box, font_path, max_pt, min_pt=14, line_spacing=0.9, serif=False, align="left", italic=False):
    """
    Binary-search the font size so the wrapped text fits in the bounding box height/width.
    Returns (wrapped_text, font).
    """
    left, top, right, bottom = box
    box_w, box_h = right - left, bottom - top
    lo, hi = min_pt, max_pt
    chosen = min_pt
    font_fallback = FALLBACK_SERIF if serif else FALLBACK_SANS
    face = font_path

    while lo <= hi:
        mid = (lo + hi) // 2
        font = load_font(face, mid, fallback=font_fallback)
        # rough wrap width: measure average char width
        avg = draw.textlength("ABCDEFGHIJKLMNOPQRSTUVWXYZ", font=font) / 26
        if avg == 0:
            avg = mid * 0.6
        # target chars per line:
        lines = wrap_text_by_width(draw, text, font, box_w)
        wrapped = "\n".join(lines)

        # measure height
        line_h = font.getbbox("Ag")[3] - font.getbbox("Ag")[1]
        total_h = int(len(lines) * line_h * line_spacing)
        # check width overflow on longest line
        max_w = max((draw.textlength(l, font=font) for l in lines), default=0)
        if total_h <= box_h and max_w <= box_w:
            chosen = mid
            lo = mid + 1
            chosen_wrapped = wrapped
            chosen_font = font
        else:
            hi = mid - 1

    return chosen_wrapped, chosen_font

def draw_text_in_box(draw, text, box, font_path, max_pt, serif=False, color="black", align="left", italic=False, line_spacing=1.05, min_pt=14):
    wrapped, font = shrink_to_fit(draw, text, box, font_path, max_pt, min_pt=min_pt, line_spacing=line_spacing, serif=serif, align=align, italic=italic)
    draw.multiline_text((box[0], box[1]), wrapped, font=font, fill=color, spacing=int(font.size * (line_spacing - 1) + 2), align=align)

def draw_mana_cost(draw, mana_str, box, font_path):
    """
    Very simple fallback: render {4}{G}{G} as text aligned right.
    For full fidelity, replace with PNG icons and place them horizontally.
    """
    font = load_font(font_path, 36, fallback=FALLBACK_SANS)
    # right-align
    text_w = draw.textlength(mana_str, font=font)
    x = box[2] - text_w
    y = box[1] + (box[3] - box[1] - (font.getbbox("Ag")[3]-font.getbbox("Ag")[1])) // 2
    draw.text((x, y), mana_str, font=font, fill="black")

# -----------------------------
# 3) COMPOSE
# -----------------------------

def render_card(card, art: Image.Image, out_path=OUTPUT_PATH):
    art_resized = art.resize((ART_BOX[2] - ART_BOX[0], ART_BOX[3] - ART_BOX[1]), Image.LANCZOS)

    # Base canvas
    if card.get("mana_cost")[-2] == "B":
        frame_path = BLACK_FRAME_PATH
        TYPE_BOX = (82, 870, 900, 910)
        TEXT_BOX = (82, 950, 900, 1345)  # rules + flavor
        RARITY_C = (900, 893)               # small circle center (title bar right)
    else:
        frame_path = GREEN_FRAME_PATH
        TYPE_BOX = (82, 860, 880, 900)
        TEXT_BOX = (82, 935, 880, 1330)
        RARITY_C = (900, 880)
    canvas = Image.new("RGBA", (W, H), (0, 0, 0, 0))
    # Place resized art (cropped to ART BOX)
    canvas.paste(art_resized, (ART_BOX[0], ART_BOX[1]))
    frame = Image.open(frame_path).convert("RGBA")
    canvas = Image.alpha_composite(canvas, frame)

    canvas.convert("RGB").save("demo_image.png", "PNG")

    # draw text on top
    draw = ImageDraw.Draw(canvas)

    # Fonts
    title_font_path = FONT_BELEREN_SC if Path(FONT_BELEREN_SC).exists() else FONT_BELEREN_B
    type_font_path  = title_font_path
    rules_font_path = FONT_RULES if Path(FONT_RULES).exists() else FALLBACK_SERIF

    # Name (SmallCaps style face), big, left-aligned
    draw_text_in_box(draw, card["name"], NAME_BOX, title_font_path, max_pt=56, serif=False)

    # Mana cost (right)
    #draw_mana_cost(draw, f"{{{card['mana_cost']}}}" if "{" not in card["mana_cost"] else card["mana_cost"], MANA_BOX, title_font_path)

    # Type line
    draw_text_in_box(draw, card["type_line"], TYPE_BOX, type_font_path, max_pt=40, serif=False)

    # Rules text: abilities (bulleted) + flavor (italic color)
    rules = "\n".join([f" {a}\n" for a in card.get("abilities", [])]) if card.get("abilities", []) else ""
    flavor = card.get("flavor_text", "").strip()

    # Split TEXT_BOX: 80% for rules, last 20% for flavor
    tL, tT, tR, tB = TEXT_BOX
    rules_h = int((tB - tT) * 0.78)
    flavor_box = (tL, tT + rules_h + 10, tR, tB)
    rules_box  = (tL, tT, tR, tT + rules_h)

    # draw rules (serif, a bit tighter)
    if rules:
        print(rules_box)
        draw_text_in_box(draw, rules, rules_box, rules_font_path, max_pt=40, serif=True, line_spacing=1.08)

    # draw flavor (italic feel—if you don’t have italic, this still works)
    if flavor:
        draw_text_in_box(draw, f"“{flavor}”", flavor_box, rules_font_path, max_pt=40, serif=True, color="#555555", line_spacing=1.06)

    # PT box
    pt_text = f"{card['power']}/{card['toughness']}"
    pt_font = load_font(FONT_BELEREN_B if Path(FONT_BELEREN_B).exists() else FALLBACK_SANS, 72, fallback=FALLBACK_SANS)
    # center PT in its box
    pt_w = draw.textlength(pt_text, font=pt_font)
    pt_h = pt_font.getbbox("Ag")[3] - pt_font.getbbox("Ag")[1]
    px = PT_BOX[0] + (PT_BOX[2] - PT_BOX[0] - pt_w) // 2
    py = PT_BOX[1] + (PT_BOX[3] - PT_BOX[1] - pt_h) // 2
    draw.text((px, py), pt_text, font=pt_font, fill="black")

    # rarity dot (simple)
    rarity_color = {"Mythic Rare": (237, 125, 49), "Rare": (212, 175, 55), "Uncommon": (160, 160, 160), "Common": (40, 40, 40)}
    rc = rarity_color.get(card.get("rarity", "Common"), (40, 40, 40))
    rx, ry = RARITY_C
    r = 16
    draw.ellipse((rx - r, ry - r, rx + r, ry + r), fill=rc, outline=(0, 0, 0))

    # save
    canvas.convert("RGB").save(f"{result_path}/{out_path}", "PNG")
    return out_path

In [7]:
out_path = render_card(card, art, out_path="mtg_card_final.png")
print("Saved:", out_path)


(82, 950, 900, 1258)
Saved: mtg_card_final.png


<img src=./datasets/MTG-card-generation/mtg_card_final.png width=300 />

What we have here is a playable MTG card which looks much like the real thing! The template could still improve somewhat, but it does look like a good mock-up.</br>
### Recap and Future Efforts
This demo shows us that AI tools can be a productivity multiplier when used as part of a larger development strategy. By creating separate agents who work together, you can create something that is not just interesting but valuable.
</br>
This only shows a small slice of MTG cards. It's certainly possible (and could be a lot of fun!) to build an entire MTG set where all cards follow an overall theme. For example, one popular existing set is based on the Lord of the Rings books and movies. However, it wouldn't be difficult to build an entire set of 100+ cards which all tie to your favorite movie. The possiblities are endless.